###Dataset

>Dataset is taken from https://www.win.tue.nl/~mpechen/projects/smm/.

>It consists of 5331 positive and 5331 negative movie reviews in Turkish.

###Load Dataset

In [ ]:
!curl https://www.win.tue.nl/~mpechen/projects/smm/Turkish_Movie_Sentiment.zip -o ./Turkish_Movie_Sentiment.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  520k  100  520k    0     0  3225k      0 --:--:-- --:--:-- --:--:-- 3234k


In [ ]:
!unzip /content/Turkish_Movie_Sentiment.zip -d /content

Archive:  /content/Turkish_Movie_Sentiment.zip
replace /content/tr_polarity.neg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

> Each line in the review files is one review.

In [ ]:
# Positive Reviews
with open("./tr_polarity.pos", "rb") as f:
  review_pos_list = f.read().decode("iso-8859-9").replace("\r", "").split("\n")

# Negative Reviews
with open("./tr_polarity.neg", "rb") as f:
  review_neg_list = f.read().decode("iso-8859-9").replace("\r", "").split("\n")

In [ ]:
print(f"First 5 positive reviews of total {len(review_pos_list)}:\n ", *review_pos_list[:5], sep="\n")
print(f"\n\nFirst 5 negative review of total {len(review_neg_list)}:\n ", *review_neg_list[:5], sep="\n")

First 5 positive reviews of total 5332:
 
gerçekten harika bir yapim birçok kez izledim gene izlerim özgürlük askini ve ingilizlerin ne kadar vahset olduklarini gözler önüne seren bir film ve tabi ki ask.... 
her izledigimde hayranlik duydugum gerçek klasik diyebilecegimiz filmlerden . içinde teknik hatalar barindirsa bile sinema olgusunun en üst noktalarindan.. 
gerçekten tarihi savas filmleri arasinda tartismasiz en iyisi , 12 yil boyunca acaba ikincisi çekirimi diye bekledigim bir film ,belki william wallace babasinin ölümünden sonra amcasi yanina almisti onu yetistirmisti belki bunu anlatan mükkemmel bir filim olablilr=). 
aldigi ödülleri sonuna dek hak eden muhtesem bir basyapit . 
özgürlük denilince aklima gelen ilk film.bir basyapit.. 


First 5 negative review of total 5332:
 
giseye oynayan bir film.mel gibson'in oyunculugu yine çok kötü.film bastan sona duygu sömürüsü ama anlayan nerde!. 
bircok yonden sahip olduklari zayifliklari populerligi iyi kullanmasiyla gidermis zayif 

In [ ]:
for idx, rev in enumerate(review_pos_list):
  if not rev:
    print(idx)

for idx, rev in enumerate(review_neg_list):
  if not rev:
    print(idx)

5331
4890
5331


We will use pandas for dataset operations using DataFrame.

In [ ]:
import pandas as pd

We will use NLTK library for text preprocessing


In [ ]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Convert positive and negative reviews to dataframe.

To do that, we first convert the reviews a list of list such that:

```
[ [1, 'review'], [1, 'review'], ..., [0, 'review'], [0, 'review'], ...]
```

In [ ]:
# Convert reviews to pandas dataframe
dataset = pd.DataFrame([[1, prev] for prev in review_pos_list if prev] + [[0, nrev] for nrev in review_neg_list if nrev] , columns=["label", "review"])

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10661 entries, 0 to 10660
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   10661 non-null  int64 
 1   review  10661 non-null  object
dtypes: int64(1), object(1)
memory usage: 166.7+ KB


Tokenize the documents

####Punctuations Stop Words Case Folding

In [ ]:
from nltk.corpus import stopwords;
nltk.download("stopwords");
stop_words = stopwords.words("turkish");

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
print(stop_words)

['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani']


In [ ]:
from nltk.tokenize import word_tokenize
import re
# Removing punctuations and stop words, and applying case folding
for idx, row in dataset.iterrows():
    dataset.at[idx, "review"] = [token.lower() for token in word_tokenize(re.sub(r"\.", " . ", row["review"]))  if token.isalpha() and (token.lower() not in stop_words)]

###Train-Test Splits

In [ ]:
# Train Dataset
train_dataset = dataset.sample(frac=0.8, random_state=1)

In [ ]:
train_dataset

,label,review
1777,1,"[iyi, film, noluyor, lan, demekten, filme, kon..."
6818,0,"[das, experimenti, yeniden, çekmisler, gerek, ..."
1305,1,"[yaw, herhangi, bir, kanal, filmi, yayinlasa, ..."
6106,0,"[filmi, dün, gece, izledim, bekledigim, çikmad..."
1185,1,"[tom, hanks, senaryo, iyi, film, zaten, asmisss]"
...,...,...
7062,0,"[acayip, zorlama, bir, film, olmus, puani, pop..."
4690,1,"[dvd, sini, nerdeyse, hafta, önce, izledim, sa..."
6497,0,"[serinin, iyi, filmiydi, serinin, ilk, filmi, ..."
10558,0,"[seyrettim, foktan, filimdi, gidip, korsanini,..."


In [ ]:
len(train_dataset)

8529

In [ ]:
train_dataset["label"].value_counts()

1    4275
0    4254
Name: label, dtype: int64

In [ ]:
# Test Dataset
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
test_dataset

,label,review
0,1,"[gerçekten, harika, bir, yapim, birçok, izledi..."
2,1,"[gerçekten, tarihi, savas, filmleri, arasinda,..."
15,1,"[önce, bir, çikarip, izledim, senedir, arada, ..."
18,1,"[iskoçyanin, evlatlari, baslayip, özgürlügü, a..."
20,1,"[kesinlikle, kaçirilmamasi, gerekne, filmlerin..."
...,...,...
10639,0,"[hafta, sonu, izledim, basrol, oyuncusunun, ka..."
10644,0,"[yakinda, apoya, film, çekilirse, sasirmam]"
10647,0,"[kendileri, inanacakki, karsiyida, inandirarak..."
10650,0,"[oldu, olacak, oscara, aday, gösterelim, filmm..."


In [ ]:
len(test_dataset)

2132

In [ ]:
test_dataset["label"].value_counts()

0    1076
1    1056
Name: label, dtype: int64

### Build Vocabulary

In [ ]:
total_tokens = 0
token_set = set()
for _, tokens in train_dataset["review"].items():
  total_tokens += len(tokens)
  for token in tokens:
    token_set.add(token)

# Add <UNKNOWN> to token set for occurence of words that are not in corpus.
token_set.add("<UNKNOWN>")

print(f"Corpus size: {total_tokens}, Vocabulary size: {len(token_set)}")
print("First 10 tokens:")

i = 0
for token in token_set:
  if i >= 10:
    break
  print(token)
  i += 1



Corpus size: 145979, Vocabulary size: 25227
First 10 tokens:
unlu
bayginn
kahkalara
seydir
irmaki
hizla
misiniz
birlestirilmis
kolik
fumba


### Vectorization of Movie Reviews:

- Once the vocabulary is established, each document is represented as a numerical vector. The length of the vector is equal to the size of the vocabulary, and each element in the vector corresponds to the count of the respective word in the document.

In [ ]:
# term-to-index mapping
term2idx = {}
for idx, term in enumerate(token_set):
  term2idx.update({term:idx})

print(f"term-to-index mapping:\nSize: {len(term2idx)}\nterms and indices:\n")
i = 0
for key in term2idx:
  if i >= 10:
    break
  print(f"{key}: {term2idx[key]}")
  i += 1

term-to-index mapping:
Size: 25227
terms and indices:

unlu: 0
bayginn: 1
kahkalara: 2
seydir: 3
irmaki: 4
hizla: 5
misiniz: 6
birlestirilmis: 7
kolik: 8
fumba: 9


In [ ]:
# Document vectors of Train Set
train_vector_df = None
vocab_size = len(token_set)
doc_vectors = []
for _, row in train_dataset.iterrows():
  doc_vector = [0] * vocab_size # initial vector
  for token in row["review"]:
    doc_vector[term2idx[token]] += 1
  doc_vectors.append([row["label"], doc_vector])

train_vector_df = pd.DataFrame(doc_vectors, columns= ["label", "vector"])

###Model Training
Here, we simply use aggregation (sum) of the review vectors for each sentiment category as the sentiment representation.

In [ ]:
import numpy as np

# Aggregated vectors for positive & negative sentiments
V = len(token_set)
pos_vector = np.array([0] * V) # Initial positive sentiment vector
neg_vector = np.array([0] * V) # Initial negative sentiment vector

for _, row in train_vector_df.iterrows():
  curr_review_vec = np.array(row['vector'])
  if row['label'] == 0:
      neg_vector += curr_review_vec
  else:
      pos_vector += curr_review_vec


In [ ]:
print(f"positive sentiment vector of size: {len(pos_vector)}\n", pos_vector[:180])
print(f"negative sentiment vector of size: {len(neg_vector)}\n", neg_vector[:180])

positive sentiment vector of size: 25227
 [ 6  1  2  1  3  1  3  1  5  1  1  1  1  0  1  1  2  0  0 30  1  2  0  6
  0  1  5  0  1  0  7  0  3  1  1  1  2  1  0  0  1  1  0  1  0  1  0  0
  0  0  1  0  1  0  0  0  1  0  1  7  0  0  1  5  0  0  0  0  0  0  0  0
  1  2  0  0  0  1  0  0  0  0  1  1  2  1  2  0  1  1  1  0  1  0  0  2
  0  0  4  0  0 12  0  0  2  0  1  5 29  1  0  0  0  6  0  1  0  1  0  1
  2  0  1  0  0  1  0  0  2  0  1  0  1  1  1  0  1  1  1  0  2  0  1  0
  0  1  0  0  1  2  0  0  1  1  1  1  9  1  1  2  1  1  4  2  1  3  1  0
  0  1  1  0  0  0  0  1  0  0  1  1]
negative sentiment vector of size: 25227
 [13  0  0  1  1  0  0  3  4  2  0  0  0  1  0  0 14  2  1 23  0  4  2  0
  1  1 10  1  0  1  0  2  2  0  0  1  0  0  1  1  2  0  2  0  1  0  1  1
  1  1  0  1  0  1  1  1  1  1  2  0  1  1  0  9  7  1  2  1  1  1  1  1
  0  3  1  1  1  0  1  1  1  1  5  1  2  0  0  2  2  0  0  1  0  1  1  1
  1  1  2  4  1  6  1  2  3  1  0  7 18  0  3  1  2  0  1  3  1  2  1  0
  

### Similarity Measure: Cosine

Using cosine measure, classify each review in the **test** dataset, such that,

* Ff the review has a cosine similarity score closer to the **pos** reviews in training dataset, then mark the review as **pos**;

* Otherwise, mark it as **neg**


In [ ]:
def cosine_sim(vector1, vector2):
  dot_product = np.dot(vector1, vector2)  # dot product of the vectors
  magnitude1 = np.linalg.norm(vector1)    # norm => length of vector1
  magnitude2 = np.linalg.norm(vector2)    # norm => length of vector2

  cosine_similarity = dot_product / (magnitude1 * magnitude2)
  return cosine_similarity

###Testing and Result

In [ ]:
# Document vectors of Test Set
test_vector_df = None
vocab_size = len(token_set)
test_doc_vectors = []
for _, row in test_dataset.iterrows():
  doc_vector = [0] * vocab_size # initial vector
  for token in row["review"]:
    doc_vector[term2idx.get(token, term2idx.get("<UNKNOWN>"))] += 1
  test_doc_vectors.append([row["label"], doc_vector])

test_vector_df = pd.DataFrame(test_doc_vectors, columns= ["label", "vector"])

In [ ]:
test_labels, test_vectors  = test_vector_df.iloc[:,0], test_vector_df.iloc[:,1]
test_labels.value_counts()
test_labels.values

array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
# make all predictions
predicted_labels = []
for vector in test_vectors:
  pos_sim = cosine_sim(np.array(vector), pos_vector)
  neg_sim = cosine_sim(np.array(vector), neg_vector)
  predicted_labels.append((1 if pos_sim > neg_sim else 0))

<ipython-input-28-5a200f9499e2>:6: RuntimeWarning: invalid value encountered in double_scalars
  cosine_similarity = dot_product / (magnitude1 * magnitude2)


In [ ]:
# Check predictions
len(predicted_labels)

2132

In [ ]:
# check if prediction made correctly or not for each prediction
total_predictions = len(predicted_labels)
correct_predictions = 0
pos_pos = 0 # predicted pos, it is pos   (correct prediction)
neg_neg = 0 # predicted neg, it is neg   (correct prediction)
pos_neg = 0 # predicted pos, it is neg   (uncorrect prediction)
neg_pos = 0 # predicted neg, it is pos   (uncorrect prediction)
for idx ,prediction in enumerate(predicted_labels):
  label = test_labels[idx]

  if prediction and label:
    pos_pos += 1
  elif prediction and not label:
    pos_neg += 1
  elif not prediction and not label:
    neg_neg += 1
  else:
    neg_pos += 1

  correct_predictions += 1 if prediction == test_labels[idx] else 0

print(f"Success rate: {correct_predictions / total_predictions:.4f}")
print(f"\nPrediction\tLabel")
print(f"positive\tpositive\t{pos_pos / total_predictions:.4f}")
print(f"negative\tnegative\t{neg_neg / total_predictions:.4f}")
print(f"positive\tnegative\t{pos_neg / total_predictions:.4f}")
print(f"negative\tpositive\t{neg_pos / total_predictions:.4f}")

Success rate: 0.8007

Prediction	Label
positive	positive	0.4029
negative	negative	0.3977
positive	negative	0.1069
negative	positive	0.0924
